In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import random
import time
import re

timeDelay = random.randrange(5, 10)

In [2]:
def products(category):
    url = "https://www.influenster.com/reviews/" + category + "/"
    
    try:
        myDriver.get(url)
        myDriver.find_element_by_xpath('//*[@id="CookielawBanner"]/div/p/a').click();
    except TimeoutException:
        print("Page expired")
        
    product_link = 'https://www.influenster.com/reviews/'
    product_list = []
    click_count = 0
    while len(product_list) < 2:
        try:
            links = [a.get_attribute('href') for a in myDriver.find_elements_by_class_name('category-product')]
            for link in links:
                if link not in product_list:
                    product_list.append(link)
                    print(link)
            
            time.sleep(timeDelay)
            load_more = myDriver.find_element_by_class_name("category-products-pagination-page.next")
            load_more.click()
            click_count+=1
            time.sleep(timeDelay)
            
        except Exception as e:
            print(e)
            break
    else:
        print(click_count)
        return product_list[:1]

In [3]:
def products_cards(category):
    url = "https://www.influenster.com/reviews/" + category + "/"
    
    try:
        myDriver.get(url)
        myDriver.find_element_by_xpath('//*[@id="CookielawBanner"]/div/p/a').click();
    except TimeoutException:
        print("Page expired")
        
    product_link = 'https://www.influenster.com/reviews/'
    product_list = []
    product_detail_list = []
    click_count = 0
    while click_count < 100: #50000
        try:
            product_template = myDriver.find_element_by_class_name('category-products')
            cards = product_template.find_elements_by_class_name('category-product')
            for card in cards:
                link = card.get_attribute('href')
                if link not in product_list:
                    product_list.append(link)
                    name = card.find_element_by_class_name('category-product-title').text
                    brand = card.find_element_by_class_name('category-product-brand').text
                    star_rating = card.find_element_by_class_name('avg-stars.small').get_attribute('data-stars')
                    reviews = card.find_element_by_class_name('category-product-rating').text
                    image = card.find_element_by_css_selector('img').get_attribute('src')
                    
                    product_detail_list.append({'product_link': link, 'product_name': name, 'brand': brand, 'star_rating': star_rating, 
                                                'total_reviews': re.findall(r'[0-9.,]+', reviews)[-1], 'image_url': image})

                    print('name:', name, 'link:', link, 'brand:', brand, 'star_rating:', 
                          star_rating, 'reviews:', re.findall(r'[0-9.,]+', reviews)[-1], 'image_url:', image)
            time.sleep(timeDelay)
            load_more = product_template.find_element_by_class_name("category-products-pagination-page.next")
            load_more.click()
            click_count+=1
            time.sleep(timeDelay)
            
        except Exception as e:
            print(e)
            return product_detail_list
            break
    else:
        print(click_count)
        return product_detail_list

In [4]:
def product_details(urls):
    """Take a post url and return post details"""
    product_detail_list = []
    for link in urls:
        print(link)
        myDriver.get(link)
        time.sleep(timeDelay)
        try:    
            #prod_link = link.replace('https://www.influenster.com/reviews/','')
            prod_link = link
        except:
            prod_link=None
            
        try:
            xpath_name = '//*[@id="app-base"]/div[1]/div[2]/div[2]/div[2]/div[1]/h1'
            product_name = myDriver.find_element_by_xpath(xpath_name).text
        except Exception as e:
            print(e)
            product_name = None
            
        try:
            total_reviews = myDriver.find_element_by_xpath('//*[@id="app-base"]/div[1]/div[2]/div[2]/div[2]/div[1]/div[2]/div[2]').text
        except Exception as e:
            print(e)
            total_reviews=None
        
        try:                     
            xpath_description = '//*[@id="app-base"]/div[1]/div[2]/div[2]/div[2]/div[2]/div'
            description = myDriver.find_element_by_xpath(xpath_description).text
        except Exception as e:
            print(e)
            description=None
            
        try:
            xpath_price = '//*[@id="app-base"]/div[1]/div[2]/div[2]/div[2]/div[1]/div[4]/a/div[1]/div[1]/strong'
            price = myDriver.find_element_by_xpath(xpath_price).text
        except Exception as e:
            print(e)
            price=None
            
        product_detail_list.append({'product_link': prod_link, 'product_name': product_name,'total_reviews': total_reviews.strip("()"),'price': price, 'description': description})
        print(prod_link,product_name,total_reviews.strip("()"),price,description)
        time.sleep(timeDelay)
        
    return product_detail_list

In [5]:
def product_reviews(url):
    print(url)
    myDriver.get(url)
    time.sleep(timeDelay)
    prod_reviews_list = []
    review_text_list = []
    click_count=0

    while (1):
        try:
            cards = []
            #product_link = url.replace('https://www.influenster.com/reviews/','')
            review_wrapper = myDriver.find_element_by_class_name("search__ReviewWrapper-sc-1ylxfoz-1.hwQBqj.layoutComponents__Block-l2otzz-0.ixyxcj")
            cards = review_wrapper.find_elements_by_class_name("item.wrappers__Wrapper-sc-1mex847-0.fNRjgH")
            
            for card in cards:
                try:
                    class_review_text = 'review-text.layoutComponents__Text-l2otzz-1.kgitTG.layoutComponents__Block-l2otzz-0.ixyxcj'
                    review_text = card.find_element_by_class_name(class_review_text).text
                except Exception as e:
                    print(e)
                if review_text not in review_text_list:
                    review_text_list.append(review_text)
                    try:
                        class_review_link = 'layoutComponents__Row-l2otzz-2.hXWPE.layoutComponents__Block-l2otzz-0.ixyxcj'
                        review_link = card.find_element_by_class_name(class_review_link).find_element_by_css_selector('a').get_attribute('href')
                    except Exception as e:
                        print(e)
                        review_link = None
                    try:
                        class_review_date = 'layoutComponents__Text-l2otzz-1.dilWrZ.layoutComponents__Block-l2otzz-0.ixyxcj'
                        review_date = card.find_element_by_class_name(class_purchased_at).text
                    except Exception as e:
                        print(e)
                        review_date = None
                    class_reviewer = 'layoutComponents__Row-l2otzz-2.MSbai.layoutComponents__Block-l2otzz-0.ixyxcj'
                    try:
                        reviewer_name = card.find_element_by_class_name(class_reviewer).find_element_by_css_selector('a').text
                    except Exception as e:
                        print(e)
                        reviewer_name = None
                    try:
                        reviewer_link = card.find_element_by_class_name(class_reviewer).find_element_by_css_selector('a').get_attribute('href')
                    except Exception as e:
                        print(e)
                        reviewer_link = None
                    try:
                        class_loc = 'author-card.authorCard__AuthorCardWrapperDesktop-sc-26la3e-1.hAEwt'
                        reviewer_loc = card.find_element_by_class_name(class_loc).find_element_by_css_selector('span').text
                    except Exception as e:
                        print(e)
                        reviewer_loc = None
                    try:
                        class_like = 'like.like__LikeWrap-sc-1om89ge-0.gAkUgg'
                        likes = card.find_element_by_class_name(class_like).text
                    except Exception as e:
                        print(e)
                        likes = None
                    prod_reviews_list.append({'review_id': re.findall(r'[0-9]+', review_link), 'review_link': review_link, 'product_link': url, 'review_text': review_text,
                                              'review_date': review_date,'reviewer_name': reviewer_name, 'reviewer_loc': reviewer_loc,
                                             'likes': re.findall(r'[0-9]+', likes), 'reviewer_link': reviewer_link})
                    print('review_id: ', re.findall(r'[0-9]+', review_link), 'review_link:', review_link, 'product_link:', url, 'review_text:', review_text,
                          'review_date:', review_date, 'reviewer_name:', reviewer_name, 'reviewer_loc:', 
                          reviewer_loc, 'likes:', re.findall(r'[0-9]+', likes), 'reviewer_link:', reviewer_link)
                    
            load_more = review_wrapper.find_element_by_css_selector('li.next')
            if load_more.get_attribute("class") == 'next':
                load_more.click()
                click_count+=1
                time.sleep(timeDelay)
            else:
                break
                
        except Exception as e:
            print(e)
            break
    else:
        print('click' + click_count) 
    return prod_reviews_list

In [6]:
try:
    # set up chromedriver
    myDriver=webdriver.Chrome()
    actions = ActionChains(myDriver)
    #prod_list = products_cards("multivitamins")
    #prod_data_frame = pd.DataFrame(prod_list)
    #prod_data_frame.to_csv("influenster_product_list_multivitamins.csv", index=False)
    prod_list = products("multivitamins")
    print(prod_list)
    #prod_details = product_details(prod_list)
    #prod_data_frame = pd.DataFrame(prod_details)
    #prod_data_frame.to_csv("influenster_product_list_dog-food.csv", index=False)
    prod_reviews = []
    for prod in prod_list:
       product_reviews_res = product_reviews(prod)
       print(product_reviews_res)
       prod_reviews_df = pd.DataFrame(product_reviews_res)
       prod_reviews_df.to_csv("influenster_product_reviews_multivitamins.csv", mode='a', index=False, header=False)

except Exception as e:
    print(e)
finally:
    myDriver.quit()
    print('exit')

https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies
https://www.influenster.com/reviews/vitafusiontm-multivitestm-complete-multivitamin-berry-peach-and-orange
https://www.influenster.com/reviews/vitafusiontm-womens-multivitamin-gummy
https://www.influenster.com/reviews/one-a-day-womens
https://www.influenster.com/reviews/centrum-multigummies-women-79
https://www.influenster.com/reviews/centrum-silver-adults
https://www.influenster.com/reviews/optimal-solutions-womens-raspberry-multivitamin-gummies-80-count
https://www.influenster.com/reviews/yummi-bears-multivitamin-mineral-gummy-bears
https://www.influenster.com/reviews/azo-bladder-control-with-go-less
https://www.influenster.com/reviews/one-a-day-womens-vitacraves-gummies
https://www.influenster.com/reviews/one-a-day-mens-health-formula
https://www.influenster.com/reviews/apex-fitness-apex-fit-multivitamin-50-plus-23-essential-vitamins-and-minerals-60-tablet-bottle
https://www.influenster.

('review_id: ', [u'56246229'], 'review_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies/56246229', 'product_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies', 'review_text:', u'These are super tasty! I can\u2019t say if they are working any better than other multis I have taken in the past, but these have a nice flavor to them. Im not giving it 5 stars because they are a bit on the pricey side. I probably will only purchase these when I can find them for a good deal. Least expensive I\u2019ve found was $12.88 at Walmart. My usual gummy multis are less than $10.', 'reviewer_name:', u'Suzanne A.', 'reviewer_loc:', u'Longwood, FL', 'likes:', [u'1'], 'reviewer_link:', u'https://www.influenster.com/suzannea5')
('review_id: ', [u'54697310'], 'review_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies/54697310', 'product_lin

('review_id: ', [u'56650235'], 'review_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies/56650235', 'product_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies', 'review_text:', u'The best women\u2019s multivitamins hands-down. I get mines from Sam\u2019s and it\u2019s always so reasonably priced. \ufffc i\u2019ve been using it for about a year now and I definitely see a difference I won\u2019t try any other vitamins because I\u2019m so in love with these. \ufffc', 'reviewer_name:', u'Narcisa M.', 'reviewer_loc:', u'Ladson , SC', 'likes:', [], 'reviewer_link:', u'https://www.influenster.com/narcisah')
('review_id: ', [u'54693754'], 'review_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies/54693754', 'product_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies', 're

('review_id: ', [u'54404837'], 'review_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies/54404837', 'product_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies', 'review_text:', u"I have had horrible luck with women multi-vitamins but THESE ARE AMAZING. they are so light on the stomach and do not get me nauseous! They have given me a small energy boost and I carnally see the difference. They also state good and because of it you don't think of taking your vitamins as a job!!!! definitely a great buy.", 'reviewer_name:', u'Sullyng C.', 'reviewer_loc:', u'Plantation, FL', 'likes:', [], 'reviewer_link:', u'https://www.influenster.com/ceba___')
('review_id: ', [u'56340697'], 'review_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies/56340697', 'product_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-mu

('review_id: ', [u'54568709'], 'review_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies/54568709', 'product_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies', 'review_text:', u'By far my favorite multi vitamin gummies. I\u2019ve tried other brands but Olly multi vitamins do the work and my hair and nails have grown using the product. It\u2019s a good price value and I\u2019m overall satisfied that I\u2019ve bought these gummies several times .', 'reviewer_name:', u'Melissa C.', 'reviewer_loc:', u'Palmhurst , TX', 'likes:', [], 'reviewer_link:', u'https://www.influenster.com/mcantu4')
('review_id: ', [u'55295176'], 'review_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies/55295176', 'product_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies', 'review_text:', u'

...

('review_id: ', [u'36090261'], 'review_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies/36090261', 'product_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies', 'review_text:', u'These are the tastiest vitamins I have tried. I actually looked forward to taking them. It costs about $15 and I felt that it was worth it.', 'reviewer_name:', u'Sara K.', 'reviewer_loc:', u'Worcester, MA', 'likes:', [], 'reviewer_link:', u'https://www.influenster.com/sarak7')
Message: no such element: Unable to locate element: {"method":"css selector","selector":"span"}
  (Session info: chrome=79.0.3945.88)

('review_id: ', [u'32743013'], 'review_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies/32743013', 'product_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies', 'review_text:', u'b

('review_id: ', [u'4703987'], 'review_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies/4703987', 'product_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies', 'review_text:', u'These vitamins taste great and are much better than swallowing a pill. It has a great array of vitamins and minerals on its list.', 'reviewer_name:', u'Karolina M.', 'reviewer_loc:', u'Glendale, NY', 'likes:', [], 'reviewer_link:', u'https://www.influenster.com/karolinamcf71')
Message: no such element: Unable to locate element: {"method":"css selector","selector":"span"}
  (Session info: chrome=79.0.3945.88)

('review_id: ', [u'55078140'], 'review_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies/55078140', 'product_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies', 'review_text:', u'Bes

('review_id: ', [u'48321211'], 'review_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies/48321211', 'product_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies', 'review_text:', u'It\u2019s a wonderful vitamin! This product has a lot of sugar in it but tastes great if you prefer the gummy vitamins.', 'reviewer_name:', u'Susie M.', 'reviewer_loc:', u'Mililani, HI', 'likes:', [u'1'], 'reviewer_link:', u'https://www.influenster.com/oriata')
('review_id: ', [u'46252123'], 'review_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies/46252123', 'product_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies', 'review_text:', u'Love these gummies. Super yummy. And the packaging is so cute! Such a good brand. They have good ones for sleep to', 'reviewer_name:', u'Bethany H.', 'r

('review_id: ', [u'41358534'], 'review_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies/41358534', 'product_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies', 'review_text:', u'These vitamins taste really good but I was disappointed to see that they lacked certain things like iron', 'reviewer_name:', u'Becky H.', 'reviewer_loc:', u'Monmouth Jct, NJ', 'likes:', [u'1'], 'reviewer_link:', u'https://www.influenster.com/beckyh5052')
('review_id: ', [u'57568432'], 'review_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies/57568432', 'product_link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies', 'review_text:', u'THE BEST! I love this brand of vitamins. They all taste amazing and I feel like they actually work!', 'reviewer_name:', u'Kennedy R.', 'reviewer_loc:', u'Portl

exit
